In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')
dataset.head()


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#Customer ID and Surname would not be relevant as features
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [5]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [6]:
#We need to encode categorical variables such as geography and gender
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [7]:
#Lets Encode gender now
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization 

In [8]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [9]:
#Standardize the features
X = pipeline.fit_transform(X)

In [10]:
#Spilt the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [11]:
#Let us create the Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [12]:
#Initialize ANN
classifier = Sequential()

In [13]:
#Add input layer and hidden layer
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [14]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [15]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [16]:
#Let us take a look at our network
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 2s - loss: 0.6308 - accuracy: 0.6994 - val_loss: 0.5543 - val_accuracy: 0.7912 - 2s/epoch - 7ms/step
Epoch 2/200
225/225 - 0s - loss: 0.5276 - accuracy: 0.7887 - val_loss: 0.4839 - val_accuracy: 0.7950 - 451ms/epoch - 2ms/step
Epoch 3/200
225/225 - 0s - loss: 0.4908 - accuracy: 0.7899 - val_loss: 0.4568 - val_accuracy: 0.7950 - 428ms/epoch - 2ms/step
Epoch 4/200
225/225 - 0s - loss: 0.4717 - accuracy: 0.7932 - val_loss: 0.4432 - val_accuracy: 0.7950 - 409ms/epoch - 2ms/step
Epoch 5/200
225/225 - 0s - loss: 0.4632 - accuracy: 0.7922 - val_loss: 0.4308 - val_accuracy: 0.7950 - 463ms/epoch - 2ms/step
Epoch 6/200
225/225 - 0s - loss: 0.4508 - accuracy: 0.7901 - val_loss: 0.4225 - val_accuracy: 0.7950 - 450ms/epoch - 2ms/step
Epoch 7/200
225/225 - 0s - loss: 0.4453 - accuracy: 0.7925 - val_loss: 0.4158 - val_accuracy: 0.7950 - 450ms/epoch - 2ms/step
Epoch 8/200
225/225 - 0s - loss: 0.4402 - accuracy: 0.7956 - val_loss: 0.4107 - val_accuracy: 0.7950 - 418ms/epoch - 2ms/

Epoch 66/200
225/225 - 1s - loss: 0.3943 - accuracy: 0.8294 - val_loss: 0.3510 - val_accuracy: 0.8525 - 590ms/epoch - 3ms/step
Epoch 67/200
225/225 - 1s - loss: 0.3934 - accuracy: 0.8275 - val_loss: 0.3506 - val_accuracy: 0.8500 - 586ms/epoch - 3ms/step
Epoch 68/200
225/225 - 1s - loss: 0.3965 - accuracy: 0.8260 - val_loss: 0.3513 - val_accuracy: 0.8500 - 746ms/epoch - 3ms/step
Epoch 69/200
225/225 - 1s - loss: 0.3978 - accuracy: 0.8256 - val_loss: 0.3506 - val_accuracy: 0.8512 - 577ms/epoch - 3ms/step
Epoch 70/200
225/225 - 0s - loss: 0.3930 - accuracy: 0.8282 - val_loss: 0.3518 - val_accuracy: 0.8512 - 414ms/epoch - 2ms/step
Epoch 71/200
225/225 - 0s - loss: 0.3936 - accuracy: 0.8283 - val_loss: 0.3523 - val_accuracy: 0.8500 - 413ms/epoch - 2ms/step
Epoch 72/200
225/225 - 0s - loss: 0.3953 - accuracy: 0.8261 - val_loss: 0.3530 - val_accuracy: 0.8487 - 403ms/epoch - 2ms/step
Epoch 73/200
225/225 - 0s - loss: 0.3948 - accuracy: 0.8297 - val_loss: 0.3504 - val_accuracy: 0.8512 - 402ms/e

Epoch 131/200
225/225 - 1s - loss: 0.3868 - accuracy: 0.8294 - val_loss: 0.3430 - val_accuracy: 0.8537 - 737ms/epoch - 3ms/step
Epoch 132/200
225/225 - 1s - loss: 0.3888 - accuracy: 0.8322 - val_loss: 0.3427 - val_accuracy: 0.8525 - 756ms/epoch - 3ms/step
Epoch 133/200
225/225 - 1s - loss: 0.3885 - accuracy: 0.8272 - val_loss: 0.3438 - val_accuracy: 0.8525 - 726ms/epoch - 3ms/step
Epoch 134/200
225/225 - 1s - loss: 0.3889 - accuracy: 0.8307 - val_loss: 0.3424 - val_accuracy: 0.8525 - 546ms/epoch - 2ms/step
Epoch 135/200
225/225 - 1s - loss: 0.3865 - accuracy: 0.8307 - val_loss: 0.3424 - val_accuracy: 0.8512 - 530ms/epoch - 2ms/step
Epoch 136/200
225/225 - 1s - loss: 0.3844 - accuracy: 0.8310 - val_loss: 0.3431 - val_accuracy: 0.8512 - 506ms/epoch - 2ms/step
Epoch 137/200
225/225 - 1s - loss: 0.3882 - accuracy: 0.8289 - val_loss: 0.3449 - val_accuracy: 0.8500 - 528ms/epoch - 2ms/step
Epoch 138/200
225/225 - 0s - loss: 0.3844 - accuracy: 0.8315 - val_loss: 0.3438 - val_accuracy: 0.8500 -

225/225 - 1s - loss: 0.3857 - accuracy: 0.8306 - val_loss: 0.3455 - val_accuracy: 0.8450 - 756ms/epoch - 3ms/step
Epoch 196/200
225/225 - 1s - loss: 0.3808 - accuracy: 0.8326 - val_loss: 0.3429 - val_accuracy: 0.8487 - 741ms/epoch - 3ms/step
Epoch 197/200
225/225 - 1s - loss: 0.3843 - accuracy: 0.8314 - val_loss: 0.3420 - val_accuracy: 0.8512 - 654ms/epoch - 3ms/step
Epoch 198/200
225/225 - 1s - loss: 0.3848 - accuracy: 0.8256 - val_loss: 0.3434 - val_accuracy: 0.8500 - 572ms/epoch - 3ms/step
Epoch 199/200
225/225 - 1s - loss: 0.3835 - accuracy: 0.8315 - val_loss: 0.3435 - val_accuracy: 0.8525 - 537ms/epoch - 2ms/step
Epoch 200/200
225/225 - 1s - loss: 0.3845 - accuracy: 0.8304 - val_loss: 0.3439 - val_accuracy: 0.8512 - 552ms/epoch - 2ms/step


In [19]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 [==============================] - 0s 3ms/step
[[0.27081585]
 [0.36896664]
 [0.21612236]
 [0.10636221]
 [0.09059718]]


In [20]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [21]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1546   49]
 [ 279  126]]


In [22]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

83.6 % of data was classified correctly
